# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), December 12, 2021**

In [1]:
using DataFrames

## Split-apply-combine

### Grouping a data frame

In [2]:
x = DataFrame(id=[1,2,3,4,1,2,3,4], id2=[1,2,1,2,1,2,1,2], v=rand(8))

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.751676
2,2,2,0.887757
3,3,1,0.272597
4,4,2,0.211121
5,1,1,0.446431
6,2,2,0.625962
7,3,1,0.0655041
8,4,2,0.734711


In [3]:
groupby(x, :id)

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.751676
2,1,1,0.446431
,id,id2,v
,Int64,Int64,Float64
1,4,2,0.211121
2,4,2,0.734711


In [4]:
groupby(x, [])

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.751676
2,2,2,0.887757
3,3,1,0.272597
4,4,2,0.211121
5,1,1,0.446431
6,2,2,0.625962
7,3,1,0.0655041
8,4,2,0.734711


In [5]:
gx2 = groupby(x, [:id, :id2])

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.751676
2,1,1,0.446431
,id,id2,v
,Int64,Int64,Float64
1,4,2,0.211121
2,4,2,0.734711


In [6]:
parent(gx2) # get the parent DataFrame 

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.751676
2,2,2,0.887757
3,3,1,0.272597
4,4,2,0.211121
5,1,1,0.446431
6,2,2,0.625962
7,3,1,0.0655041
8,4,2,0.734711


In [7]:
vcat(gx2...) # back to the DataFrame, but in a different order of rows than the original

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.751676
2,1,1,0.446431
3,2,2,0.887757
4,2,2,0.625962
5,3,1,0.272597
6,3,1,0.0655041
7,4,2,0.211121
8,4,2,0.734711


In [8]:
DataFrame(gx2) # the same

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.751676
2,1,1,0.446431
3,2,2,0.887757
4,2,2,0.625962
5,3,1,0.272597
6,3,1,0.0655041
7,4,2,0.211121
8,4,2,0.734711


In [9]:
DataFrame(gx2, keepkeys=false) # drop grouping columns when creating a data frame

,v
,Float64
1,0.751676
2,0.446431
3,0.887757
4,0.625962
5,0.272597
6,0.0655041
7,0.211121
8,0.734711


In [10]:
groupcols(gx2) # vector of names of grouping variables

2-element Vector{Symbol}:
 :id
 :id2

In [11]:
valuecols(gx2) # and non-grouping variables

1-element Vector{Symbol}:
 :v

In [12]:
groupindices(gx2) # group indices in parent(gx2)

8-element Vector{Union{Missing, Int64}}:
 1
 2
 3
 4
 1
 2
 3
 4

In [13]:
kgx2 = keys(gx2)

4-element DataFrames.GroupKeys{GroupedDataFrame{DataFrame}}:
 GroupKey: (id = 1, id2 = 1)
 GroupKey: (id = 2, id2 = 2)
 GroupKey: (id = 3, id2 = 1)
 GroupKey: (id = 4, id2 = 2)

You can index into a `GroupedDataFrame` like to a vector or to a dictionary.
The second form acceps `GroupKey`, `NamedTuple` or a `Tuple`

In [14]:
gx2

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.751676
2,1,1,0.446431
,id,id2,v
,Int64,Int64,Float64
1,4,2,0.211121
2,4,2,0.734711


In [15]:
k = keys(gx2)[1]

GroupKey: (id = 1, id2 = 1)

In [16]:
ntk = NamedTuple(k)

(id = 1, id2 = 1)

In [17]:
tk = Tuple(k)

(1, 1)

the operations below produce the same result and are fast

In [18]:
gx2[1]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.751676
2,1,1,0.446431


In [19]:
gx2[k]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.751676
2,1,1,0.446431


In [20]:
gx2[ntk]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.751676
2,1,1,0.446431


In [21]:
gx2[tk]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.751676
2,1,1,0.446431


handling missing values

In [22]:
x = DataFrame(id = [missing, 5, 1, 3, missing], x = 1:5)

,id,x
,Int64?,Int64
1,missing,1
2,5,2
3,1,3
4,3,4
5,missing,5


In [23]:
groupby(x, :id) # by default groups include mising values and their order is not guaranteed

,id,x
,Int64?,Int64
1,1,3
,id,x
,Int64?,Int64
1,missing,1
2,missing,5


In [24]:
groupby(x, :id, sort=true, skipmissing=true) # but we can change it; now they are sorted

,id,x
,Int64?,Int64
1,1,3
,id,x
,Int64?,Int64
1,5,2


In [25]:
groupby(x, :id, sort=false) # and now they are in the order they appear in the source data frame

,id,x
,Int64?,Int64
1,missing,1
2,missing,5
,id,x
,Int64?,Int64
1,3,4


### Performing transformations by group using `combine`, `select`, `select!`, `transform`, and `transform!`

In [26]:
using Statistics
using Chain

In [27]:
ENV["LINES"] = 15 # reduce the number of rows in the output

15

In [28]:
x = DataFrame(id=rand('a':'d', 100), v=rand(100))

,id,v
,Char,Float64
1,b,0.579621
2,a,0.894544
3,c,0.492607
4,d,0.616283
5,b,0.714875
6,b,0.493625
7,b,0.636216
8,a,0.999821
9,c,0.811971


In [29]:
# apply a function to each group of a data frame
# combine keeps as many rows as are returned from the function
@chain x begin
    groupby(:id)
    combine(:v => mean)
end

,id,v_mean
,Char,Float64
1,b,0.529711
2,a,0.624415
3,c,0.507397
4,d,0.497843


In [30]:
x.id2 = axes(x, 1)

Base.OneTo(100)

In [31]:
# select and transform keep as many rows as are in the source data frame and in correct order
# additionally transform keeps all columns from the source
@chain x begin
    groupby(:id)
    transform(:v => mean)
end

,id,v,id2,v_mean
,Char,Float64,Int64,Float64
1,b,0.579621,1,0.529711
2,a,0.894544,2,0.624415
3,c,0.492607,3,0.507397
4,d,0.616283,4,0.497843
5,b,0.714875,5,0.529711
6,b,0.493625,6,0.529711
7,b,0.636216,7,0.529711
8,a,0.999821,8,0.624415
9,c,0.811971,9,0.507397


In [32]:
# note that combine reorders rows by group of GroupedDataFrame
@chain x begin
    groupby(:id)
    combine(:id2, :v => mean)
end

,id,id2,v_mean
,Char,Int64,Float64
1,b,1,0.529711
2,b,5,0.529711
3,b,6,0.529711
4,b,7,0.529711
5,b,11,0.529711
6,b,13,0.529711
7,b,16,0.529711
8,b,24,0.529711
9,b,25,0.529711


In [33]:
# we give a custom name for the result column
@chain x begin
    groupby(:id)
    combine(:v => mean => :res)
end

,id,res
,Char,Float64
1,b,0.529711
2,a,0.624415
3,c,0.507397
4,d,0.497843


In [34]:
# you can have multiple operations
@chain x begin
    groupby(:id)
    combine(:v => mean => :res1, :v => sum => :res2, nrow => :n)
end

,id,res1,res2,n
,Char,Float64,Float64,Int64
1,b,0.529711,12.7131,24
2,a,0.624415,14.3616,23
3,c,0.507397,11.6701,23
4,d,0.497843,14.9353,30


Additional notes:
* `select!` and `transform!` perform operations in-place
* The general syntax for transformation is `source_columns => function => target_column`
* if you pass multiple columns to a function they are treated as positional arguments
* `ByRow` and `AsTable` work exactly like discussed for operations on data frames in 05_columns.ipynb
* you can automatically groupby again the result of `combine`, `select` etc. by passing `ungroup=false` keyword argument to them
* similarly `keepkeys` keyword argument allows you to drop grouping columns from the resulting data frame

It is also allowed to pass a function to all these functions (also - as a special case, as a first argument). In this case the return value can be a table. In particular it allows for an easy dropping of groups if you return an empty table from the function.

If you pass a function you can use a `do` block syntax. In case of passing a function it gets a `SubDataFrame` as its argument.

Here is an example:

In [35]:
combine(groupby(x, :id)) do sdf
    n = nrow(sdf)
    n < 25 ? DataFrame() : DataFrame(n=n) # drop groups with low number of rows
end

,id,n
,Char,Int64
1,d,30


You can also produce multiple columns in a single operation, e.g.:

In [36]:
df = DataFrame(id=[1,1,2,2], val=[1,2,3,4])

,id,val
,Int64,Int64
1,1,1
2,1,2
3,2,3
4,2,4


In [37]:
@chain df begin
    groupby(:id)
    combine(:val => (x -> [x]) => AsTable)
end

,id,x1,x2
,Int64,Int64,Int64
1,1,1,2
2,2,3,4


In [38]:
@chain df begin
    groupby(:id)
    combine(:val => (x -> [x]) => [:c1, :c2])
end

,id,c1,c2
,Int64,Int64,Int64
1,1,1,2
2,2,3,4


It is easy to unnest the column into multiple columns, e.g.

In [39]:
df = DataFrame(a=[(p=1, q=2), (p=3, q=4)])

,a
,NamedTup…
1,"(p = 1, q = 2)"
2,"(p = 3, q = 4)"


In [40]:
select(df, :a => AsTable)

,p,q
,Int64,Int64
1,1,2
2,3,4


In [41]:
df = DataFrame(a=[[1, 2], [3, 4]])

,a
,Array…
1,"[1, 2]"
2,"[3, 4]"


In [42]:
select(df, :a => AsTable) # automatic column names generated

,x1,x2
,Int64,Int64
1,1,2
2,3,4


In [43]:
select(df, :a => [:C1, :C2]) # custom column names generated

,C1,C2
,Int64,Int64
1,1,2
2,3,4


Finally, observe that one can conveniently apply multiple transformations using broadcasting:

In [44]:
df = DataFrame(id=repeat(1:10, 10), x1=1:100, x2=101:200)

,id,x1,x2
,Int64,Int64,Int64
1,1,1,101
2,2,2,102
3,3,3,103
4,4,4,104
5,5,5,105
6,6,6,106
7,7,7,107
8,8,8,108
9,9,9,109


In [45]:
@chain df begin
    groupby(:id)
    combine([:x1, :x2] .=> minimum)
end

,id,x1_minimum,x2_minimum
,Int64,Int64,Int64
1,1,1,101
2,2,2,102
3,3,3,103
4,4,4,104
5,5,5,105
6,6,6,106
7,7,7,107
8,8,8,108
9,9,9,109


In [46]:
@chain df begin
    groupby(:id)
    combine([:x1, :x2] .=> [minimum maximum])
end

,id,x1_minimum,x2_minimum,x1_maximum,x2_maximum
,Int64,Int64,Int64,Int64,Int64
1,1,1,101,91,191
2,2,2,102,92,192
3,3,3,103,93,193
4,4,4,104,94,194
5,5,5,105,95,195
6,6,6,106,96,196
7,7,7,107,97,197
8,8,8,108,98,198
9,9,9,109,99,199


### Aggregation of a data frame using `mapcols`

In [47]:
x = DataFrame(rand(10, 10), :auto)

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.110505,0.965201,0.331423,0.937123,0.138926,0.911185,0.770002,0.214889
2,0.786783,0.696481,0.107579,0.857731,0.185325,0.10649,0.42865,0.0791703
3,0.384666,0.17547,0.507479,0.765465,0.781298,0.174763,0.531973,0.82144
4,0.742875,0.534935,0.524817,0.923702,0.98908,0.852975,0.337276,0.811552
5,0.505054,0.0742155,0.330711,0.701384,0.467586,0.0275657,0.241088,0.765081
6,0.872524,0.968185,0.94927,0.698919,0.15139,0.647034,0.328877,0.797069
7,0.0192887,0.619312,0.853525,0.584465,0.679903,0.576564,0.977158,0.810688
8,0.670286,0.846774,0.583749,0.684217,0.0776146,0.604345,0.445025,0.364179
9,0.280338,0.342745,0.93098,0.184023,0.560151,0.683693,0.825101,0.23244


In [48]:
mapcols(mean, x)

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.50888,0.610862,0.581908,0.634715,0.415212,0.4866,0.564262,0.570237,0.605061


### Mapping rows and columns using `eachcol` and `eachrow`

In [49]:
map(mean, eachcol(x)) # map a function over each column and return a vector

10-element Vector{Float64}:
 0.5088797494143702
 0.6108619179105791
 0.581907842918649
 0.6347151784261741
 0.4152119972212532
 0.48660038738308364
 0.5642623575103447
 0.5702367028988784
 0.6050605606487032
 0.5045073692757193

In [50]:
# an iteration returns a Pair with column name and values
foreach(c -> println(c[1], ": ", mean(c[2])), pairs(eachcol(x)))

x1: 0.5088797494143702
x2: 0.6108619179105791
x3: 0.581907842918649
x4: 0.6347151784261741
x5: 0.4152119972212532
x6: 0.48660038738308364
x7: 0.5642623575103447
x8: 0.5702367028988784
x9: 0.6050605606487032
x10: 0.5045073692757193


In [51]:
# now the returned value is DataFrameRow which works as a NamedTuple but is a view to a parent DataFrame
map(r -> r.x1/r.x2, eachrow(x))

10-element Vector{Float64}:
 0.11448872527970738
 1.1296554444601636
 2.1921998834884002
 1.3887217637513376
 6.805242890770891
 0.901194756902887
 0.031145401059764443
 0.7915764873352583
 0.8179202067247894
 0.8093040493799194

In [52]:
# it prints like a data frame, only the caption is different so that you know the type of the object
er = eachrow(x)

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.110505,0.965201,0.331423,0.937123,0.138926,0.911185,0.770002,0.214889
2,0.786783,0.696481,0.107579,0.857731,0.185325,0.10649,0.42865,0.0791703
3,0.384666,0.17547,0.507479,0.765465,0.781298,0.174763,0.531973,0.82144
4,0.742875,0.534935,0.524817,0.923702,0.98908,0.852975,0.337276,0.811552
5,0.505054,0.0742155,0.330711,0.701384,0.467586,0.0275657,0.241088,0.765081
6,0.872524,0.968185,0.94927,0.698919,0.15139,0.647034,0.328877,0.797069
7,0.0192887,0.619312,0.853525,0.584465,0.679903,0.576564,0.977158,0.810688
8,0.670286,0.846774,0.583749,0.684217,0.0776146,0.604345,0.445025,0.364179
9,0.280338,0.342745,0.93098,0.184023,0.560151,0.683693,0.825101,0.23244


In [53]:
er.x1 # you can access columns of a parent data frame directly

10-element Vector{Float64}:
 0.11050461215670915
 0.7867831342553142
 0.3846660923064339
 0.7428753028428708
 0.5050543031364177
 0.872523561430998
 0.01928873418323085
 0.6702863442193413
 0.28033826199693035
 0.7164771476154571

In [54]:
# it prints like a data frame, only the caption is different so that you know the type of the object
ec = eachcol(x)

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.110505,0.965201,0.331423,0.937123,0.138926,0.911185,0.770002,0.214889
2,0.786783,0.696481,0.107579,0.857731,0.185325,0.10649,0.42865,0.0791703
3,0.384666,0.17547,0.507479,0.765465,0.781298,0.174763,0.531973,0.82144
4,0.742875,0.534935,0.524817,0.923702,0.98908,0.852975,0.337276,0.811552
5,0.505054,0.0742155,0.330711,0.701384,0.467586,0.0275657,0.241088,0.765081
6,0.872524,0.968185,0.94927,0.698919,0.15139,0.647034,0.328877,0.797069
7,0.0192887,0.619312,0.853525,0.584465,0.679903,0.576564,0.977158,0.810688
8,0.670286,0.846774,0.583749,0.684217,0.0776146,0.604345,0.445025,0.364179
9,0.280338,0.342745,0.93098,0.184023,0.560151,0.683693,0.825101,0.23244


In [55]:
ec.x1 # you can access columns of a parent data frame directly

10-element Vector{Float64}:
 0.11050461215670915
 0.7867831342553142
 0.3846660923064339
 0.7428753028428708
 0.5050543031364177
 0.872523561430998
 0.01928873418323085
 0.6702863442193413
 0.28033826199693035
 0.7164771476154571

### Transposing

you can transpose a data frame using `permutedims`:

In [56]:
df = DataFrame(reshape(1:12, 3, 4), :auto)

,x1,x2,x3,x4
,Int64,Int64,Int64,Int64
1,1,4,7,10
2,2,5,8,11
3,3,6,9,12


In [57]:
df.names = ["a", "b", "c"]

3-element Vector{String}:
 "a"
 "b"
 "c"

In [58]:
permutedims(df, :names)

,names,a,b,c
,String,Int64,Int64,Int64
1,x1,1,2,3
2,x2,4,5,6
3,x3,7,8,9
4,x4,10,11,12
